In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from scipy.special import softmax

In [2]:
df1 = pd.read_csv("../ex36_resnet152_bert-base-japanese-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6426
df2 = pd.read_csv("../ex37_resnet152_bert-base-japanese-whole-word-masking_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6454
df3 = pd.read_csv("../ex39_swin_b_bert-base-japanese-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6407
df4 = pd.read_csv("../ex40_vit_b_16_bert-base-japanese-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6514
df5 = pd.read_csv("../ex43_swin_b_sentence-bert-base-ja-mean-tokens-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6458
df6 = pd.read_csv("../ex42_efficientnet_v2_m_bert-base-japanese-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6439
df7 = pd.read_csv("../ex45_swin_b_bert-base-japanese-char-v2/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6485
df8 = pd.read_csv("../ex48_clip_vit_base_patch32_bert-base-japanese-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6454
df9 = pd.read_csv("../ex50_clip-vit-b-32-japanese-v1_sentence-bert-base-ja-mean-tokens-v2_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 
df10 = pd.read_csv("../ex54_swin_b_t5-base-japanese-v1.1_7folds/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6574
df11 = pd.read_csv("../ex58_swin_b_bert-base-japanese-v2_7folds_pseudo/oof_df.csv").sort_values("id").reset_index(drop=True) # cv: 0.6503

In [3]:
df1.head()

,id,odai_photo_file_name,text,is_laugh,img_path,fold,pred_0,pred_1
0,005fsegda,gfwqgf5nr3.jpg,結婚式の馴れ初めムービーが太陽系の形成から始まった,1,../bokete/train/gfwqgf5nr3.jpg,5,-0.11835,-0.19350
1,00bxdcozk,xjdx5fqpmv.jpg,ホームレスだったが『なかなかの雰囲気をかもし出してる』という理由で料理長になれた。,0,../bokete/train/xjdx5fqpmv.jpg,3,-0.72900,0.27170
2,00mpolpif,f09vzizm3w.jpg,見慣れない旅人が、たくさんの壺を割りながら近づいてくる,1,../bokete/train/f09vzizm3w.jpg,0,-0.56050,0.99270
3,00xnsh2c6,5eutjv8q0f.jpg,「押すなよ、ほんっとうに押すなよ！押すなって！押すなああぁぁぁ………」,0,../bokete/train/5eutjv8q0f.jpg,1,-0.06744,0.05313
4,01fvui4tv,udoedxn436.jpg,あまり頭の良くない息子が唯一合格した私立地獄谷デーモン幼稚園の送迎バス。,1,../bokete/train/udoedxn436.jpg,1,-0.44400,0.47920


In [4]:
df2.head()

,id,odai_photo_file_name,text,is_laugh,img_path,fold,pred_0,pred_1
0,005fsegda,gfwqgf5nr3.jpg,結婚式の馴れ初めムービーが太陽系の形成から始まった,1,../bokete/train/gfwqgf5nr3.jpg,5,0.3154,0.09950
1,00bxdcozk,xjdx5fqpmv.jpg,ホームレスだったが『なかなかの雰囲気をかもし出してる』という理由で料理長になれた。,0,../bokete/train/xjdx5fqpmv.jpg,3,-0.4443,0.61330
2,00mpolpif,f09vzizm3w.jpg,見慣れない旅人が、たくさんの壺を割りながら近づいてくる,1,../bokete/train/f09vzizm3w.jpg,0,-0.1813,0.51070
3,00xnsh2c6,5eutjv8q0f.jpg,「押すなよ、ほんっとうに押すなよ！押すなって！押すなああぁぁぁ………」,0,../bokete/train/5eutjv8q0f.jpg,1,0.1859,0.04147
4,01fvui4tv,udoedxn436.jpg,あまり頭の良くない息子が唯一合格した私立地獄谷デーモン幼稚園の送迎バス。,1,../bokete/train/udoedxn436.jpg,1,-0.3896,0.74170


In [6]:
df11.head()

,id,odai_photo_file_name,text,is_laugh,img_path,fold,pred_0,pred_1
0,005fsegda,gfwqgf5nr3.jpg,結婚式の馴れ初めムービーが太陽系の形成から始まった,1,../bokete/train/gfwqgf5nr3.jpg,5,0.440860,0.564339
1,00bxdcozk,xjdx5fqpmv.jpg,ホームレスだったが『なかなかの雰囲気をかもし出してる』という理由で料理長になれた。,0,../bokete/train/xjdx5fqpmv.jpg,3,0.185504,0.796292
2,00mpolpif,f09vzizm3w.jpg,見慣れない旅人が、たくさんの壺を割りながら近づいてくる,1,../bokete/train/f09vzizm3w.jpg,0,0.171200,0.821044
3,00xnsh2c6,5eutjv8q0f.jpg,「押すなよ、ほんっとうに押すなよ！押すなって！押すなああぁぁぁ………」,0,../bokete/train/5eutjv8q0f.jpg,1,0.454351,0.545346
4,01fvui4tv,udoedxn436.jpg,あまり頭の良くない息子が唯一合格した私立地獄谷デーモン幼稚園の送迎バス。,1,../bokete/train/udoedxn436.jpg,1,0.289071,0.720171


In [24]:
preds1 = softmax(df1[["pred_0", "pred_1"]].values, axis=-1)
preds2 = softmax(df2[["pred_0", "pred_1"]].values, axis=-1)
preds3 = softmax(df3[["pred_0", "pred_1"]].values, axis=-1)
preds4 = softmax(df4[["pred_0", "pred_1"]].values, axis=-1)
preds5 = softmax(df5[["pred_0", "pred_1"]].values, axis=-1)
preds6 = softmax(df6[["pred_0", "pred_1"]].values, axis=-1)
preds7 = softmax(df7[["pred_0", "pred_1"]].values, axis=-1)
preds8 = softmax(df8[["pred_0", "pred_1"]].values, axis=-1)
preds9 = softmax(df9[["pred_0", "pred_1"]].values, axis=-1)
preds10 = softmax(df10[["pred_0", "pred_1"]].values, axis=-1)
preds11 = df11[["pred_0", "pred_1"]].values

In [25]:
val_preds = (preds1 * 0.5 + preds2 * 0.5 + preds3 * 0.5 + preds4 * 0.5 + preds5 * 0.5 + preds6 * 0.5 + preds7 * 0.5 + preds8 * 0.5 + preds9 * 0.5 + preds10 * 0.5 + preds11 * 0.5) / 5.5

print(np.sum(val_preds[:3], axis=-1))
log_loss(df1["is_laugh"].values, val_preds)

[1.00047264 0.99834514 0.9992949 ]


0.6345164261333307

In [26]:
from scipy.optimize import minimize

def optimize_weights(x):
    val_preds = (preds1 * x[0] + preds2 * x[1] + preds3 * x[2] + preds4 * x[3] + preds5 * x[4] + preds6 * x[5] + preds7 * x[6] + preds8 * x[7] + preds9 * x[8] + preds10 * x[9] + preds11 * x[10]) / sum(x)

    score = log_loss(df1["is_laugh"].values, val_preds)
    return score

n = 11
weight_init = [1 / n for _ in range(n)]
result = minimize(optimize_weights,
                  weight_init,
                  method="Nelder-Mead",
                  # bounds=bounds, 
                  # options={"maxiter": 1}
                 )
print(f'result:')
print("[" + ", ".join(map(str, result.x)) + "]")

x = result.x

val_preds = (preds1 * x[0] + preds2 * x[1] + preds3 * x[2] + preds4 * x[3] + preds5 * x[4] + preds6 * x[5] + preds7 * x[6] + preds8 * x[7] + preds9 * x[8] + preds10 * x[9] + preds11 * x[10]) / sum(x)

score = log_loss(df1["is_laugh"].values, val_preds)
print(score)

result:
[0.1563895842636911, 0.12886919692284887, 0.1272798953877381, -0.10879046484437135, 0.11290727962681868, 0.143645226925178, 0.09298438929818265, 0.10802328164805639, 0.05511603300194007, -0.1260642854238916, 0.2885010553284587]
0.6321426662110509


In [27]:
np.sum(val_preds[:3], axis=-1)

array([1.00153231, 0.99463486, 0.99771402])

In [28]:
df = df1.copy()
df[["pred_0", "pred_1"]] = val_preds
df.to_csv("oof_df_ensemble.csv", index=False)

In [29]:
from scipy.optimize import minimize

weights = x

preds_eval = (preds1 * weights[0] + preds2 * weights[1] + preds3 * weights[2] + preds4 * weights[3]
             + preds5 * weights[4] + preds6 * weights[5] + preds7 * weights[6] + preds8 * weights[7] 
             + preds9 * weights[8] + preds10 * weights[9] + preds11 * weights[10] 
        )



def optimize_weights(x):
    preds = preds_eval.copy()[:, 1].reshape(-1, 1)
    
    preds[preds < 0.1] = preds[preds < 0.1] * x[0]
    preds[(0.1 <= preds) & (preds < 0.2)] = preds[(0.1 <= preds) & (preds < 0.2)] * x[1]
    preds[(0.2 <= preds) & (preds < 0.3)] = preds[(0.2 <= preds) & (preds < 0.3)] * x[2]
    preds[(0.3 <= preds) & (preds < 0.4)] = preds[(0.3 <= preds) & (preds < 0.4)] * x[3]
    preds[(0.4 <= preds) & (preds < 0.5)] = preds[(0.4 <= preds) & (preds < 0.5)] * x[4]
    preds[(0.5 <= preds) & (preds < 0.6)] = preds[(0.5 <= preds) & (preds < 0.6)] * x[5]
    preds[(0.6 <= preds) & (preds < 0.7)] = preds[(0.6 <= preds) & (preds < 0.7)] * x[6]
    preds[(0.7 <= preds) & (preds < 0.8)] = preds[(0.7 <= preds) & (preds < 0.8)] * x[7]
    preds[(0.8 <= preds) & (preds < 0.9)] = preds[(0.8 <= preds) & (preds < 0.9)] * x[8]
    preds[(0.9 <= preds) & (preds <= 1.0)] = preds[(0.9 <= preds) & (preds <= 1.0)] * x[9]
    
    predictions = np.zeros((preds.shape[0], 2))
    predictions[:, 1] = preds.reshape(-1)
    predictions[:, 0] = 1.0 - preds.reshape(-1)
    
    score = log_loss(df1["is_laugh"].values, predictions)
    return score

weight_init = [1.0 for _ in range(10)]
result = minimize(optimize_weights,
                  weight_init,
                  method="Nelder-Mead",
                  # bounds=bounds, 
                  # options={"maxiter": 1}
                 )
print(f'result:')
print("[" + ", ".join(map(str, result.x)) + "]")

weights1 = result.x
preds = preds_eval.copy()[:, 1].reshape(-1, 1)

preds[preds < 0.1] = preds[preds < 0.1] * x[0]
preds[(0.1 <= preds) & (preds < 0.2)] = preds[(0.1 <= preds) & (preds < 0.2)] * weights1[1]
preds[(0.2 <= preds) & (preds < 0.3)] = preds[(0.2 <= preds) & (preds < 0.3)] * weights1[2]
preds[(0.3 <= preds) & (preds < 0.4)] = preds[(0.3 <= preds) & (preds < 0.4)] * weights1[3]
preds[(0.4 <= preds) & (preds < 0.5)] = preds[(0.4 <= preds) & (preds < 0.5)] * weights1[4]
preds[(0.5 <= preds) & (preds < 0.6)] = preds[(0.5 <= preds) & (preds < 0.6)] * weights1[5]
preds[(0.6 <= preds) & (preds < 0.7)] = preds[(0.6 <= preds) & (preds < 0.7)] * weights1[6]
preds[(0.7 <= preds) & (preds < 0.8)] = preds[(0.7 <= preds) & (preds < 0.8)] * weights1[7]
preds[(0.8 <= preds) & (preds < 0.9)] = preds[(0.8 <= preds) & (preds < 0.9)] * weights1[8]
preds[(0.9 <= preds) & (preds <= 1.0)] = preds[(0.9 <= preds) & (preds <= 1.0)] * weights1[9]
# preds[5.0 < preds] = 5.0
print(preds[:5])

predictions = np.zeros((preds.shape[0], 2))
predictions[:, 1] = preds.reshape(-1)
predictions[:, 0] = 1.0 - preds.reshape(-1)
print(predictions[:5])
score = log_loss(df1["is_laugh"].values, predictions)
print(score)

result:
[0.9848579021928361, 1.0411295557791602, 0.90462887642677, 1.0204667352782182, 1.0064829842787586, 1.0148418685731189, 1.0026343665382378, 1.0044571788841692, 1.0235264911438613, 1.0164582996478846]
[[0.53673643]
 [0.71932269]
 [0.77694697]
 [0.52775769]
 [0.72037448]]
[[0.46326357 0.53673643]
 [0.28067731 0.71932269]
 [0.22305303 0.77694697]
 [0.47224231 0.52775769]
 [0.27962552 0.72037448]]
0.6317221680728933


In [30]:
def optimize_weights(x):
    preds = preds_eval.copy()[:, 1].reshape(-1, 1)
    
    preds[preds < 0.05] = preds[preds < 0.05] * x[0]
    preds[(0.05 <= preds) & (preds < 0.1)] = preds[(0.05 <= preds) & (preds < 0.1)] * x[1]
    preds[(0.1 <= preds) & (preds < 0.15)] = preds[(0.1 <= preds) & (preds < 0.15)] * x[2]
    preds[(0.15 <= preds) & (preds < 0.2)] = preds[(0.15 <= preds) & (preds < 0.2)] * x[3]
    preds[(0.2 <= preds) & (preds < 0.25)] = preds[(0.2 <= preds) & (preds < 0.25)] * x[4]
    preds[(0.25 <= preds) & (preds < 0.3)] = preds[(0.25 <= preds) & (preds < 0.3)] * x[5]
    preds[(0.3 <= preds) & (preds < 0.35)] = preds[(0.3 <= preds) & (preds < 0.35)] * x[6]
    preds[(0.35 <= preds) & (preds < 0.4)] = preds[(0.35 <= preds) & (preds < 0.4)] * x[7]
    preds[(0.4 <= preds) & (preds < 0.45)] = preds[(0.4 <= preds) & (preds < 0.45)] * x[8]
    preds[(0.45 <= preds) & (preds <= 0.5)] = preds[(0.45 <= preds) & (preds <= 0.5)] * x[9]
    preds[(0.5 <= preds) & (preds <= 0.55)] = preds[(0.5 <= preds) & (preds <= 0.55)] * x[10]
    preds[(0.55 <= preds) & (preds <= 0.6)] = preds[(0.55 <= preds) & (preds <= 0.6)] * x[11]
    preds[(0.6 <= preds) & (preds <= 0.65)] = preds[(0.6 <= preds) & (preds <= 0.65)] * x[12]
    preds[(0.65 <= preds) & (preds <= 0.7)] = preds[(0.65 <= preds) & (preds <= 0.7)] * x[13]
    preds[(0.7 <= preds) & (preds <= 0.75)] = preds[(0.7 <= preds) & (preds <= 0.75)] * x[14]
    preds[(0.75 <= preds) & (preds <= 0.8)] = preds[(0.75 <= preds) & (preds <= 0.8)] * x[15]
    preds[(0.8 <= preds) & (preds <= 0.85)] = preds[(0.8 <= preds) & (preds <= 0.85)] * x[16]
    preds[(0.85 <= preds) & (preds <= 0.9)] = preds[(0.85 <= preds) & (preds <= 0.9)] * x[17]
    preds[(0.9 <= preds) & (preds <= 0.95)] = preds[(0.9 <= preds) & (preds <= 0.95)] * x[18]
    preds[(0.95 <= preds) & (preds <= 1.0)] = preds[(0.95 <= preds) & (preds <= 1.0)] * x[19]
    
    predictions = np.zeros((preds.shape[0], 2))
    predictions[:, 1] = preds.reshape(-1)
    predictions[:, 0] = 1.0 - preds.reshape(-1)
    
    score = log_loss(df1["is_laugh"].values, predictions)
    return score

weight_init = [1.0 for _ in range(20)]
result = minimize(optimize_weights,
                  weight_init,
                  method="Nelder-Mead",
                  # bounds=bounds, 
                  # options={"maxiter": 1}
                 )
print(f'result:')
print("[" + ", ".join(map(str, result.x)) + "]")

weights2 = result.x
preds = preds_eval.copy()[:, 1].reshape(-1, 1)

preds[preds < 0.05] = preds[preds < 0.05] * weights2[0]
preds[(0.05 <= preds) & (preds < 0.1)] = preds[(0.05 <= preds) & (preds < 0.1)] * weights2[1]
preds[(0.1 <= preds) & (preds < 0.15)] = preds[(0.1 <= preds) & (preds < 0.15)] * weights2[2]
preds[(0.15 <= preds) & (preds < 0.2)] = preds[(0.15 <= preds) & (preds < 0.2)] * weights2[3]
preds[(0.2 <= preds) & (preds < 0.25)] = preds[(0.2 <= preds) & (preds < 0.25)] * weights2[4]
preds[(0.25 <= preds) & (preds < 0.3)] = preds[(0.25 <= preds) & (preds < 0.3)] * weights2[5]
preds[(0.3 <= preds) & (preds < 0.35)] = preds[(0.3 <= preds) & (preds < 0.35)] * weights2[6]
preds[(0.35 <= preds) & (preds < 0.4)] = preds[(0.35 <= preds) & (preds < 0.4)] * weights2[7]
preds[(0.4 <= preds) & (preds < 0.45)] = preds[(0.4 <= preds) & (preds < 0.45)] * weights2[8]
preds[(0.45 <= preds) & (preds <= 0.5)] = preds[(0.45 <= preds) & (preds <= 0.5)] * weights2[9]
preds[(0.5 <= preds) & (preds <= 0.55)] = preds[(0.5 <= preds) & (preds <= 0.55)] * weights2[10]
preds[(0.55 <= preds) & (preds <= 0.6)] = preds[(0.55 <= preds) & (preds <= 0.6)] * weights2[11]
preds[(0.6 <= preds) & (preds <= 0.65)] = preds[(0.6 <= preds) & (preds <= 0.65)] * weights2[12]
preds[(0.65 <= preds) & (preds <= 0.7)] = preds[(0.65 <= preds) & (preds <= 0.7)] * weights2[13]
preds[(0.7 <= preds) & (preds <= 0.75)] = preds[(0.7 <= preds) & (preds <= 0.75)] * weights2[14]
preds[(0.75 <= preds) & (preds <= 0.8)] = preds[(0.75 <= preds) & (preds <= 0.8)] * weights2[15]
preds[(0.8 <= preds) & (preds <= 0.85)] = preds[(0.8 <= preds) & (preds <= 0.85)] * weights2[16]
preds[(0.85 <= preds) & (preds <= 0.9)] = preds[(0.85 <= preds) & (preds <= 0.9)] * weights2[17]
preds[(0.9 <= preds) & (preds <= 0.95)] = preds[(0.9 <= preds) & (preds <= 0.95)] * weights2[18]
preds[(0.95 <= preds) & (preds <= 1.0)] = preds[(0.95 <= preds) & (preds <= 1.0)] * weights2[19]
# preds[5.0 < preds] = 5.0
print(preds[:5])

predictions = np.zeros((preds.shape[0], 2))
predictions[:, 1] = preds.reshape(-1)
predictions[:, 0] = 1.0 - preds.reshape(-1)
print(predictions[:5])
score = log_loss(df1["is_laugh"].values, predictions)
print(score)

result:
[0.9858744789933889, 0.9817776955478545, 0.9805695980605817, 0.989455121077101, 0.9386140478074081, 0.9151708933522276, 1.0362921550568833, 1.0127993568465383, 1.0132827077943012, 0.9979148122678851, 1.0238874919396057, 1.0171857941320943, 1.0121586481239686, 1.0047646776091868, 1.0076915449289685, 1.0087142225126318, 1.0059523211549983, 1.042905591461368, 1.0716767586319502, 1.0124334920145723]
[[0.54152054]
 [0.72163892]
 [0.78023979]
 [0.53246177]
 [0.72269409]]
[[0.45847946 0.54152054]
 [0.27836108 0.72163892]
 [0.21976021 0.78023979]
 [0.46753823 0.53246177]
 [0.27730591 0.72269409]]
0.6316053687912104


In [31]:
sub1 = pd.read_csv("../ex36_resnet152_bert-base-japanese-v2_7folds/submission.csv")
sub2 = pd.read_csv("../ex37_resnet152_bert-base-japanese-whole-word-masking_7folds/submission.csv")
sub3 = pd.read_csv("../ex39_swin_b_bert-base-japanese-v2_7folds/submission.csv")
sub4 = pd.read_csv("../ex40_vit_b_16_bert-base-japanese-v2_7folds/submission.csv")
sub5 = pd.read_csv("../ex43_swin_b_sentence-bert-base-ja-mean-tokens-v2_7folds/submission.csv")
sub6 = pd.read_csv("../ex42_efficientnet_v2_m_bert-base-japanese-v2_7folds/submission.csv")
sub7 = pd.read_csv("../ex45_swin_b_bert-base-japanese-char-v2/submission.csv")
sub8 = pd.read_csv("../ex48_clip_vit_base_patch32_bert-base-japanese-v2_7folds/submission.csv")
sub9 = pd.read_csv("../ex50_clip-vit-b-32-japanese-v1_sentence-bert-base-ja-mean-tokens-v2_7folds/submission.csv")
sub10 = pd.read_csv("../ex54_swin_b_t5-base-japanese-v1.1_7folds/submission.csv")
sub11 = pd.read_csv("../ex58_swin_b_bert-base-japanese-v2_7folds_pseudo/submission.csv")

In [32]:
sub1.head()

,id,is_laugh
0,rfdjcfsqq,0.474152
1,tsgqmfpef,0.380052
2,owjcthkz2,0.299579
3,rvgaocjyy,0.333452
4,uxtwu5i69,0.810307


In [33]:
sub2.head()

,id,is_laugh
0,rfdjcfsqq,0.481526
1,tsgqmfpef,0.382368
2,owjcthkz2,0.366095
3,rvgaocjyy,0.330248
4,uxtwu5i69,0.817949


In [34]:
def postprocess(preds, x):
    preds[preds < 0.05] = preds[preds < 0.05] * x[0]
    preds[(0.05 <= preds) & (preds < 0.1)] = preds[(0.05 <= preds) & (preds < 0.1)] * x[1]
    preds[(0.1 <= preds) & (preds < 0.15)] = preds[(0.1 <= preds) & (preds < 0.15)] * x[2]
    preds[(0.15 <= preds) & (preds < 0.2)] = preds[(0.15 <= preds) & (preds < 0.2)] * x[3]
    preds[(0.2 <= preds) & (preds < 0.25)] = preds[(0.2 <= preds) & (preds < 0.25)] * x[4]
    preds[(0.25 <= preds) & (preds < 0.3)] = preds[(0.25 <= preds) & (preds < 0.3)] * x[5]
    preds[(0.3 <= preds) & (preds < 0.35)] = preds[(0.3 <= preds) & (preds < 0.35)] * x[6]
    preds[(0.35 <= preds) & (preds < 0.4)] = preds[(0.35 <= preds) & (preds < 0.4)] * x[7]
    preds[(0.4 <= preds) & (preds < 0.45)] = preds[(0.4 <= preds) & (preds < 0.45)] * x[8]
    preds[(0.45 <= preds) & (preds <= 0.5)] = preds[(0.45 <= preds) & (preds <= 0.5)] * x[9]
    preds[(0.5 <= preds) & (preds <= 0.55)] = preds[(0.5 <= preds) & (preds <= 0.55)] * x[10]
    preds[(0.55 <= preds) & (preds <= 0.6)] = preds[(0.55 <= preds) & (preds <= 0.6)] * x[11]
    preds[(0.6 <= preds) & (preds <= 0.65)] = preds[(0.6 <= preds) & (preds <= 0.65)] * x[12]
    preds[(0.65 <= preds) & (preds <= 0.7)] = preds[(0.65 <= preds) & (preds <= 0.7)] * x[13]
    preds[(0.7 <= preds) & (preds <= 0.75)] = preds[(0.7 <= preds) & (preds <= 0.75)] * x[14]
    preds[(0.75 <= preds) & (preds <= 0.8)] = preds[(0.75 <= preds) & (preds <= 0.8)] * x[15]
    preds[(0.8 <= preds) & (preds <= 0.85)] = preds[(0.8 <= preds) & (preds <= 0.85)] * x[16]
    preds[(0.85 <= preds) & (preds <= 0.9)] = preds[(0.85 <= preds) & (preds <= 0.9)] * x[17]
    preds[(0.9 <= preds) & (preds <= 0.95)] = preds[(0.9 <= preds) & (preds <= 0.95)] * x[18]
    preds[(0.95 <= preds) & (preds <= 1.0)] = preds[(0.95 <= preds) & (preds <= 1.0)] * x[19]
    
    return preds

In [35]:
sub = sub1.copy()
preds = (
    (sub1["is_laugh"] * x[0] 
    + sub2["is_laugh"] * x[1]
    + sub3["is_laugh"] * x[2] 
    + sub4["is_laugh"] * x[3]
    + sub5["is_laugh"] * x[4]
    + sub6["is_laugh"] * x[5]
    + sub7["is_laugh"] * x[6]
    + sub8["is_laugh"] * x[7]
    + sub9["is_laugh"] * x[8]
    + sub10["is_laugh"] * x[9]
    + sub11["is_laugh"] * x[10]
    ) / sum(x)
    )
preds = postprocess(preds, weights2)
sub["is_laugh"] = preds
sub.to_csv("submission.csv", index=False)

In [12]:
# sub = sub1.copy()
# sub["is_laugh"] = (
#     sub1["is_laugh"] * 0.5
#     + sub2["is_laugh"] * 0.5
#     + sub3["is_laugh"] * 0.5 
#     + sub4["is_laugh"] * 0.5
#     + sub5["is_laugh"] * 0.5
#     + sub6["is_laugh"] * 0.5
#     + sub7["is_laugh"] * 0.5
#     ) / (0.5 * 7)
# sub.to_csv("submission_avg.csv", index=False)